In [1]:
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

Using TensorFlow backend.


In [2]:
df = pd.read_csv('test.csv')
df.head(10)

,tweet_text,sentiment,sentiment_confidence_summary
0,:Hello Japan is a nuclear power plant crisis. ...,Negative,"""Neutral / author is just sharing information""..."
1,. Renewable Energy Consumption Tops Nuclear fo...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
2,. Will liberals now seek to eliminate dangerou...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
3,"(Aug 22,2011)Plant Status of Fukushima Daiichi...",Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
4,[ebook] Nuclear Energy and the Environment: Ox...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
5,NaN,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
6,[news] Nuclear Power Plant: Radioactive Tritiu...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
7,[science] Nuclear Energy and the Environment: ...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
8,{link} :P How many windmills would you need to...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
9,{link} &quot;The right thing to do&quot; I'm f...,Positive,"""Neutral / author is just sharing information""..."


In [3]:
'Linhas: {} | Colunas: {}'.format(df.shape[0], df.shape[1])

'Linhas: 190 | Colunas: 3'

In [4]:
df = df.dropna()
df.head(10)

,tweet_text,sentiment,sentiment_confidence_summary
0,:Hello Japan is a nuclear power plant crisis. ...,Negative,"""Neutral / author is just sharing information""..."
1,. Renewable Energy Consumption Tops Nuclear fo...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
2,. Will liberals now seek to eliminate dangerou...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
3,"(Aug 22,2011)Plant Status of Fukushima Daiichi...",Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
4,[ebook] Nuclear Energy and the Environment: Ox...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
6,[news] Nuclear Power Plant: Radioactive Tritiu...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
7,[science] Nuclear Energy and the Environment: ...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
8,{link} :P How many windmills would you need to...,Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."
9,{link} &quot;The right thing to do&quot; I'm f...,Positive,"""Neutral / author is just sharing information""..."
10,"{link} 6 min Video: Masao Yoshida, of Fukushim...",Neutral / author is just sharing information,"""Neutral / author is just sharing information""..."


In [5]:
'Linhas: {} | Colunas: {}'.format(df.shape[0], df.shape[1])

'Linhas: 189 | Colunas: 3'

In [6]:
list(df)

['tweet_text', 'sentiment', 'sentiment_confidence_summary']

In [7]:
df['sentiment'][df['sentiment'] == 'Positive'] = 1
df['sentiment'][df['sentiment'] == 'Negative'] = -1
df['sentiment'][df['sentiment'] == 'Neutral / author is just sharing information'] = 0

In [8]:
df.head(10)

,tweet_text,sentiment,sentiment_confidence_summary
0,:Hello Japan is a nuclear power plant crisis. ...,-1,"""Neutral / author is just sharing information""..."
1,. Renewable Energy Consumption Tops Nuclear fo...,0,"""Neutral / author is just sharing information""..."
2,. Will liberals now seek to eliminate dangerou...,0,"""Neutral / author is just sharing information""..."
3,"(Aug 22,2011)Plant Status of Fukushima Daiichi...",0,"""Neutral / author is just sharing information""..."
4,[ebook] Nuclear Energy and the Environment: Ox...,0,"""Neutral / author is just sharing information""..."
6,[news] Nuclear Power Plant: Radioactive Tritiu...,0,"""Neutral / author is just sharing information""..."
7,[science] Nuclear Energy and the Environment: ...,0,"""Neutral / author is just sharing information""..."
8,{link} :P How many windmills would you need to...,0,"""Neutral / author is just sharing information""..."
9,{link} &quot;The right thing to do&quot; I'm f...,1,"""Neutral / author is just sharing information""..."
10,"{link} 6 min Video: Masao Yoshida, of Fukushim...",0,"""Neutral / author is just sharing information""..."


In [9]:
X = df['tweet_text']
Y = df['sentiment']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2)

In [10]:
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

Loaded dataset with 151 training samples, 38 test samples


In [11]:
print('---review---')
print(X_train[6])
print('---label---')
print(Y_train[6])

---review---
[news] Nuclear Power Plant: Radioactive Tritium Contamination in Vermont {link} #nwo
---label---
0


In [12]:
def gen_encode_decode(phrase):
    bag_words = set(phrase.split())
    
    dic_encode = dict([*map(lambda t: t[::-1], enumerate(bag_words))])
    dic_decode = dict([*map(lambda item: item[::-1], dic_encode.items())])
    return dic_encode, dic_decode


encode, decode = gen_encode_decode(" ".join(df['tweet_text'].values.tolist()))

In [13]:
import operator

In [14]:
df['tweet_text'].apply(lambda phrase: [*map(encode.get, phrase.split())])

0        [513, 531, 1296, 38, 438, 402, 1223, 903, 1009]
1      [559, 1240, 1324, 199, 586, 1145, 1328, 704, 1...
2      [559, 1071, 868, 49, 790, 250, 1294, 126, 438,...
3      [889, 915, 1286, 418, 677, 756, 1145, 292, 113...
4      [1293, 1145, 1324, 643, 1179, 633, 1353, 94, 1...
6      [409, 1145, 292, 537, 1355, 1326, 1001, 1109, ...
7      [435, 1145, 1324, 643, 1179, 633, 1353, 94, 10...
8      [1009, 10, 621, 854, 1047, 1239, 533, 91, 250,...
9      [1009, 310, 735, 166, 250, 771, 105, 1328, 438...
10     [1009, 945, 543, 429, 527, 881, 418, 677, 1312...
11     [1009, 1149, 131, 1230, 309, 164, 418, 438, 40...
12     [1009, 793, 399, 1147, 955, 509, 250, 438, 118...
13     [1009, 359, 1055, 110, 157, 418, 1179, 1308, 1...
14            [1009, 573, 948, 939, 1145, 292, 599, 311]
15     [1009, 573, 948, 939, 1145, 292, 599, 311, 797...
16     [1009, 565, 958, 1154, 430, 515, 418, 1086, 43...
17     [1009, 983, 250, 952, 476, 245, 926, 1328, 767...
18     [1009, 1145, 292, 871, 1

In [15]:
embedding_size=32
model=Sequential()
model.add(Embedding(df, embedding_size, input_length=500))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

TypeError: Failed to convert object of type <class 'tuple'> to Tensor. Contents: (                                            tweet_text sentiment  \
0    :Hello Japan is a nuclear power plant crisis. ...        -1   
1    . Renewable Energy Consumption Tops Nuclear fo...         0   
2    . Will liberals now seek to eliminate dangerou...         0   
3    (Aug 22,2011)Plant Status of Fukushima Daiichi...         0   
4    [ebook] Nuclear Energy and the Environment: Ox...         0   
6    [news] Nuclear Power Plant: Radioactive Tritiu...         0   
7    [science] Nuclear Energy and the Environment: ...         0   
8    {link} :P How many windmills would you need to...         0   
9    {link} &quot;The right thing to do&quot; I'm f...         1   
10   {link} 6 min Video: Masao Yoshida, of Fukushim...         0   
11   {link} 74 percent favor gradual reduction of n...         0   
12   {link} Coal seen as possible alternative to nu...         0   
13   {link} Experts are not sure o